## Goal

Generate a baby name given the first two charaters

### Create data

In [1]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM, BatchNormalization, Dropout, Bidirectional, GRU
import pandas as pd

/home/tyler/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/home/tyler/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tyler/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tyler/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/tyler/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
## number of characters to use as input
input_length = 12
## append to all names at the end, so can learn when to stop generating characters
end_name_token = "</name>"

In [3]:
lines = pd.read_csv("ds_tweets.csv")['0'].values
lines_clean = []
for line in lines:
        words = []
        for word in line.split():
            if word.startswith("#"):
                continue
            else:
                words.append(word.strip())
        line = " ".join(words)
        lines_clean.append(line.lower().strip())

In [4]:
lines_clean[15]

'comprehensive explanation to (gradient-boosting) in simplified manner.learn the concepts and implement on your own dataset. https://t.co/kosj9wpqth'

In [5]:
## create lookup dictionaries so each unique character gets a unique number
unique_chars = list(set([char for tweet in lines_clean for char in tweet]))
unique_chars = unique_chars + [end_name_token]

In [6]:
unique_chars[:2]

['💓', '|']

In [7]:
print(len(unique_chars))

252


In [8]:
char2index = {c: i for i, c in enumerate(unique_chars)}
index2char = {i: c for i, c in enumerate(unique_chars)}

In [9]:
## convert names to their index values
names_indexes = []
for name in lines_clean:
    index_name = []
    for c in name:
        index_name.append(char2index[c])
    index_name.append(char2index[end_name_token])
    names_indexes.append(index_name)

In [10]:
len(lines_clean[0])

154

In [11]:
len(names_indexes[0])

155

In [12]:
''.join([index2char[i] for i in names_indexes[0]])

'ultrasoc extends tools offering with integrated multi-core debug, visualization and data science / analytics suite read more here: https://t.co/4qpuioeydy</name>'

In [13]:
## create X and y data for training
X = []
y = []
for n in names_indexes:
    length_name = len(n)
    for i in range(0, length_name-input_length):
        X.append(n[i:i+input_length])
        y.append(n[i+input_length])
X = np.array(X)
y = np.expand_dims(np.array(y),1)

In [14]:
X.shape

(1681501, 12)

In [15]:
y.shape

(1681501, 1)

## Model

In [16]:
## we have very little data, so let's make a smalle model
vocab_size = len(unique_chars)
n_fac = 4
n_hidden = 50

In [17]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=input_length),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True),
        LSTM(n_hidden),
        Dense(vocab_size, activation='softmax'),
    ])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 12, 4)             1008      
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 4)             16        
_________________________________________________________________
lstm_1 (LSTM)                (None, 12, 50)            11000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 252)               12852     
Total params: 45,076
Trainable params: 45,068
Non-trainable params: 8
_________________________________________________________________


In [19]:
model.fit(X, y, batch_size=128, epochs=25)

Epoch 1/25
1681501/1681501 [==============================] - 331s - loss: 2.1745   
Epoch 2/25
1681501/1681501 [==============================] - 331s - loss: 1.7512   
Epoch 3/25
1681501/1681501 [==============================] - 331s - loss: 1.6308   
Epoch 4/25
1681501/1681501 [==============================] - 331s - loss: 1.5661   
Epoch 5/25
1681501/1681501 [==============================] - 330s - loss: 1.5230   
Epoch 6/25
1681501/1681501 [==============================] - 329s - loss: 1.4927   
Epoch 7/25
1681501/1681501 [==============================] - 328s - loss: 1.4698   
Epoch 8/25
1681501/1681501 [==============================] - 328s - loss: 1.4514   
Epoch 9/25
1681501/1681501 [==============================] - 328s - loss: 1.4369   
Epoch 10/25
1681501/1681501 [==============================] - 326s - loss: 1.4247   
Epoch 11/25
1681501/1681501 [==============================] - 326s - loss: 1.4145   
Epoch 12/25
1681501/1681501 [==============================] - 

In [24]:
## some helper functions to generate names!

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def name_from_two_starting_letters(two_char_string, temperature=1.0):
    two_chars_indexes = []
    generated_name = two_char_string
    for c in two_char_string:
        two_chars_indexes.append(char2index[c])
    predict_input_array = np.array([two_chars_indexes])
    predicted_index = -1
    while (predicted_index != char2index[end_name_token]):
        predicted_index = sample(model.predict(predict_input_array)[0], temperature)
        if predicted_index != char2index[end_name_token]:
            generated_name = generated_name + index2char[predicted_index]
            predict_input_array = np.expand_dims((np.append(predict_input_array[0][1:], [predicted_index])), 0)
    return generated_name

In [183]:
name_from_two_starting_letters('machine lear', temperature=0.50)

/home/tyler/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


'machine learning for the faster of the more the here, be for the better with and model providity https://t.co/hwhd4hvdh3'

In [148]:
def generate_n_random_names(n, possible_starts=['ab', 'ma', 'as', 'ch', 'em', 'cl'],
                           possible_temperatures = [0.2, 0.5, 1.0, 1.2]):
    random_names = []
    for _ in range(n):
        temperatue = np.random.choice(possible_temperatures, 1)
        starting_chars = ''.join(np.random.choice(possible_starts,1))
        random_names.append(name_from_two_starting_letters(starting_chars, temperatue))
    return random_names

In [23]:
random_names = generate_n_random_names(10, ['az'], [0.5])
for name in random_names:
    print(name)

ValueError: Error when checking : expected embedding_1_input to have shape (None, 12) but got array with shape (1, 2)